# Extração de dados do TMDB (The Movie Database)
---
27/09/2025

Este notebook tem como objetivo construir um dataset de filmes utilizando a API do The Movie Database (TMDB). O processo é realizado em duas etapas interligadas:
1. A primeira etapa é a extração dos dados de **índice** dos filmes a partir do endpoint `/discover/movie` ou do arquivo Json disponibilizado pelo TMDB e armazenado no repositório em `bronze/movie_ids_10_10_2025.json`. O método usado nesse notebook é ler diretamente do arquivo json.
2. A segunda etapa consiste em uma série de requisições para o endpoint `/movie/{movie_id}`, que retorna os dados detalhados de cada filme

###### Todos os endpoints utilizados nessa extração estão detalhados na [documentação oficial da API do TMDB](https://developer.themoviedb.org/reference/intro/getting-started)

In [2]:
import requests
import json
import time
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, explode_outer, arrays_zip, col, trim, ltrim, lit, count
from os import environ as env
from concurrent.futures import ThreadPoolExecutor, as_completed


# API credentials config -----------------------------------------------------------------------------------------------------
API_AUTH_CONFIG = {
    "tmdb_api_token": env.get("TMDB_API_TOKEN") or "tmdb_api_token_default",
    "tmdb_api_key": env.get("TMDB_API_KEY") or "tmdb_api_key_default"
}

# Api rate limit config
MAX_REQUESTS_PER_SECOND = 50
MIN_TIME_PER_REQUEST = 1.0 / MAX_REQUESTS_PER_SECOND

# Pandas configs
pd.set_option('display.max_colwidth', None)

# Create spark session
spark = SparkSession.builder.appName("movieDataIngestion").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/10/10 15:13:31 WARN Utils: Your hostname, papercut-vaio, resolves to a loopback address: 127.0.1.1; using 192.168.0.76 instead (on interface wlo1)
25/10/10 15:13:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/10 15:13:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Passo 1. Leitura dos dados de índices de filmes

| Componente | Detalhe | Finalidade |
| :--- | :--- | :--- |
| **Origem** | **`bronze/movie_ids_10_10_2025.json`** | Obtém a lista de filmes que se enquadram em critérios amplos (ex: por popularidade, gênero, ano de lançamento). |
| **Output** | Lista de **`id`** (Identificador único do filme) | O ID é o dado chave que será utilizado na próxima etapa. |

In [3]:
# Read TMDB index json file 
indexes_file_path = "bronze/movie_ids_10_10_2025.json"
df_index = spark.read.format("json").option("multiline", "false").load(indexes_file_path)

In [ ]:
# Certify no adult movie in dataset
print("Verificando se não existem filmes adultos. Valores distintos de adult:")
df_index.select("adult").distinct().show(truncate=False)

df_index.show(truncate=False)
print(f"Quantidade de índices de filmes: {df_index.count()}")

# Indexes full list for movie requests
indexes_full_list = df_index.select("id").rdd.flatMap(lambda x: x).collect()
print(f"Quantidade de itens na lista: {len(indexes_full_list)}")

Verificando se não existem filmes adultos. Valores distintos de adult:
+-----+
|adult|
+-----+
|false|
+-----+

+-----+-----+----------------------------------+----------+-----+
|adult|id   |original_title                    |popularity|video|
+-----+-----+----------------------------------+----------+-----+
|false|3924 |Blondie                           |0.435     |false|
|false|6124 |Der Mann ohne Namen               |0.5297    |false|
|false|8773 |L'Amour à vingt ans               |4.3632    |false|
|false|25449|New World Disorder 9: Never Enough|0.0852    |false|
|false|31975|Sesame Street: Elmo Loves You!    |0.0214    |true |
|false|2    |Ariel                             |1.5549    |false|
|false|3    |Varjoja paratiisissa              |2.5798    |false|
|false|5    |Four Rooms                        |3.0552    |false|
|false|6    |Judgment Night                    |3.8854    |false|
|false|8    |Life in Loops (A Megacities RMX)  |1.0294    |false|
|false|9    |Sonntag im August

Quantidade de itens na lista: 1112974
[3924, 6124, 8773, 25449, 31975, 2, 3, 5]


### Passo 2: Detalhamento Individual dos Filmes 

| Componente | Detalhe | Finalidade |
| :--- | :--- | :--- |
| **Endpoint** | **`/movie/{movie_id}`** | Realiza uma consulta específica para cada `id` obtido no Passo 1. |
| **Processo** | O código **itera** sobre a lista de IDs extraída. | Garante que cada filme tenha seus dados detalhados coletados. |
| **Output Final** | Dados detalhados (`budget`, `revenue`, `runtime`, `production_companies`, etc.). | Cria o *dataset* final para a análise. |

In [ ]:
# Set current search movie id list from index table response
movie_ids = indexes_full_list

# Function for single movie request
def fetch_film(movie_id):
    detail_url = f'https://api.themoviedb.org/3/movie/{movie_id}'
    
    params = {
        "api_key": API_AUTH_CONFIG['tmdb_api_key'],
        # "append_to_response": "genres"
    }
    
    movie_detail_response = requests.get(detail_url, params=params)
    movie_detail_string = movie_detail_response.content.decode('utf-8')
    movie_detail_json = json.loads(movie_detail_string)
    
    print(f"Fetched: {movie_detail_json['title']}")
    
    return movie_detail_json

# Function for parallel movie detail requests
def fetch_movies_parallel(movie_ids, max_workers=5):
    movie_detail_results = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {}
        
        for movie in movie_ids:
            start_time = time.time()
            
            future = executor.submit(fetch_film, movie)
            futures[future] = movie
            
            time_spent = time.time() - start_time
            sleep_time = MIN_TIME_PER_REQUEST - time_spent
            
            if sleep_time > 0:
                time.sleep(sleep_time)
        
        for future in as_completed(futures):
            try:
                movie_detail_results.append(future.result())
            except Exception as e:
                mid = futures[future]
                print(f"Erro ao buscar filme {mid}: {e}")
    return movie_detail_results

movie_detail_data = fetch_movies_parallel(movie_ids)
print(f"\nTotal de filmes buscados: {len(movie_detail_data)}")

### Passo 3. Gerar as dataframes finais para a camada bronze

Alguns dos campos retornados são arrays de objetos. Esses irão se tornar dataframes separadas, interligadas pelo ID do filme. Em seguida cada uma dessas dataframes, se tornarão um csv para a camada bronze (raw data).

Os campos identificados como array são:
1. `genres`
2. `production_companies`
3. `production_countries`
4. `spoken_languages`

Nesse contexto, os CSVs de saída dessa extração serão:
1. movies.csv
2. genres.csv
3. production_companies.csv
4. production_countries.csv
5. spoken_languages.csv

A chave extrangeira que une cada um desses artefatos, é o ID do filme. Para que isso seja garantido, será adicionada uma coluna `movie_id` em cada nos demais CSVs (com exceção de movies.csv por ser o artefato "base").

In [6]:
array_fields = ["genres", "production_companies", "production_countries", "spoken_languages"]
regular_keys = [key for key in movie_detail_data[0] if key not in array_fields]

regular_movie_fields = []

for movie in movie_detail_data:
    current_movie = {k: v for k, v in movie.items() if k not in array_fields}
    regular_movie_fields.append(current_movie)
    
movie_detail_df = pd.DataFrame(regular_movie_fields)


In [7]:
array_fields_df = {}

for key in array_fields:
    current_df = pd.json_normalize(
        data=movie_detail_data,
        record_path=key,
        meta=['id'],
        record_prefix=f'{key}_'
    )
    current_df = current_df.rename(columns={'id': 'movie_id', f'{key}_id': 'id'})
    array_fields_df[f'{key}_df'] = current_df
    
    
genres_df = array_fields_df['genres_df']
production_companies_df = array_fields_df['production_companies_df']
production_countries_df = array_fields_df['production_countries_df']
spoken_languages_df = array_fields_df['spoken_languages_df']

### Passo 4. Salvar os CSVs na camada bronze

Nesse momento existem 5 dataframes, que serão salvas em arquivos CSVs na camada bronze da seguinte forma:

|Dataframe|Descrição|Arquivo de saída|
|---|---|---|
|movie_detail_df|Dataframe principal com os dados de detalhes de filmes|movies.csv|
|genres_df|Dataframe com dados de gênero dos filmes|genres.csv|
|production_companies_df|Dataframe com dados de produtoras de filmes|production_companies.csv|
|production_countries_df|Dataframe com dados de países da produção dos filmes|production_countries.csv|
|spoken_languages_df|Dataframe com os idiomas falados para cada filme|spoken_languages.csv|

In [8]:
movie_detail_df.to_csv('bronze/movies.csv', index=False)
genres_df.to_csv('bronze/genres.csv', index=False)
production_companies_df.to_csv('bronze/production_companies.csv', index=False)
production_countries_df.to_csv('bronze/production_countries.csv', index=False)
spoken_languages_df.to_csv('bronze/spoken_languages.csv', index=False)

## 3. Ferramentas e Bibliotecas utilizadas

| Ferramenta | Uso no Notebook |
| :--- | :--- |
| **Python `requests`** | Responsável por todas as chamadas HTTP para a API do TMDB. |
| **JSON** | Manipulação e *parsing* das respostas da API, que são formatadas em JSON. |
| **Pandas** | Estruturação e armazenamento dos dados extraídos em formato DataFrame. |